In [19]:
import yaml
import torch
import time
import numpy as np

from models import EdgeGNN, GAT, GCN, GINE, SWEGNN
from data import EdgeRegressionDataset, TemporalGraphDataset

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
np.random.seed(42)
torch.manual_seed(42)

In [22]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)

In [23]:
dataset, info = TemporalGraphDataset(node_features=config['node_features'],
                    edge_features=config['edge_features'],
                    **config['dataset_parameters']).load()

print(dataset[0])
print(type(dataset[0].x), dataset[0].x.shape)
print(type(dataset[0].edge_index), dataset[0].edge_index.shape)
print(type(dataset[0].edge_attr), dataset[0].edge_attr.shape)
print(type(dataset[0].y), dataset[0].y.shape)
print(info)

Data(x=[1268, 6], edge_index=[2, 5224], edge_attr=[5224, 8], y=[1268, 1], pos=[2, 1268])
<class 'torch.Tensor'> torch.Size([1268, 6])
<class 'torch.Tensor'> torch.Size([2, 5224])
<class 'torch.Tensor'> torch.Size([5224, 8])
<class 'torch.Tensor'> torch.Size([1268, 1])
{'num_static_node_features': 3, 'num_dynamic_node_features': 1, 'num_static_edge_features': 5, 'num_dynamic_edge_features': 1, 'previous_timesteps': 2}


In [24]:
num_train = int(len(dataset) * 0.8) # 80% train, 20% test

train_dataset = dataset[:num_train]
# train_loader = DataLoader(train_dataset) # batch_size=32, shuffle=True

test_dataset = dataset[num_train:]
# test_loader = DataLoader(test_dataset) # batch_size=32, shuffle=True

In [25]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
base_model_params = {
    'static_node_features': info['num_static_node_features'],
    'dynamic_node_features': info['num_dynamic_node_features'],
    'static_edge_features': info['num_static_edge_features'],
    'dynamic_edge_features': info['num_dynamic_edge_features'],
    'previous_timesteps': info['previous_timesteps'],
    'device': device,
}
lr_info = config['training_parameters']
model_info = config['model_parameters']

In [26]:
def train(model, dataset, loss_func, optimizer):
    start_time = time.time()
    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for graph in dataset:
            graph = graph.to(device)
            labels = graph.y

            optimizer.zero_grad()

            outputs = model(graph)

            loss = loss_func(outputs, labels)
            running_loss += loss.item()

            loss.backward()
            optimizer.step()

        epoch_loss = running_loss / num_train
        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {epoch_loss:.4f}')
    end_time = time.time()
    print(f'Total training time: {end_time - start_time} seconds')


def test(model, dataset, loss_func):
    start_time = time.time()
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for graph in dataset:
            graph = graph.to(device)
            labels = graph.y

            outputs = model(graph)

            loss = loss_func(outputs, labels)
            running_loss += loss.item()
    end_time = time.time()

    # Print validation statistics
    print(f'Validation Loss: {running_loss:.4f}')
    print(f'Inference time: {end_time - start_time} seconds')

## Node Prediction Models

In [9]:
gcn_params = model_info['GCN']
model = GCN(**gcn_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()

train(model, train_dataset, loss_func, optimizer)

Epoch [1/10], Training Loss: 5.0672
Epoch [2/10], Training Loss: 0.0413
Epoch [3/10], Training Loss: 0.0209
Epoch [4/10], Training Loss: 0.0211
Epoch [5/10], Training Loss: 0.0207
Epoch [6/10], Training Loss: 0.0207
Epoch [7/10], Training Loss: 0.0207
Epoch [8/10], Training Loss: 0.0207
Epoch [9/10], Training Loss: 0.0208
Epoch [10/10], Training Loss: 0.0207
Total training time: 19.32339310646057 seconds


In [10]:
test(model, test_dataset, loss_func)

Validation Loss: 0.1839
Inference time: 0.22182083129882812 seconds


In [11]:
gat_params = model_info['GAT']
model = GAT(**gat_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()

train(model, train_dataset, loss_func, optimizer)

Epoch [1/10], Training Loss: 3.8108
Epoch [2/10], Training Loss: 0.0242
Epoch [3/10], Training Loss: 0.0215
Epoch [4/10], Training Loss: 0.0218
Epoch [5/10], Training Loss: 0.0217
Epoch [6/10], Training Loss: 0.0208
Epoch [7/10], Training Loss: 0.0207
Epoch [8/10], Training Loss: 0.0213
Epoch [9/10], Training Loss: 59.1192
Epoch [10/10], Training Loss: 0.0644
Total training time: 23.829550743103027 seconds


In [12]:
test(model, test_dataset, loss_func)

Validation Loss: 2.8843
Inference time: 0.2222757339477539 seconds


In [13]:
swe_gnn_params = model_info['SWEGNN']
model = SWEGNN(**swe_gnn_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()

train(model, train_dataset, loss_func, optimizer)

Epoch [1/10], Training Loss: 0.0988
Epoch [2/10], Training Loss: 0.0211
Epoch [3/10], Training Loss: 0.0208
Epoch [4/10], Training Loss: 0.0206


KeyboardInterrupt: 

In [ ]:
test(model, test_dataset, loss_func)

Validation Loss: 0.1826
Inference time: 1.183168649673462 seconds


In [ ]:
# No encoder decoder
swe_gnn_params = model_info['SWEGNN']
swe_gnn_params['encoder_layers'] = 0
swe_gnn_params['encoder_activation'] = None
swe_gnn_params['decoder_layers'] = 0
swe_gnn_params['decoder_activation'] = None

model = SWEGNN(**swe_gnn_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()

train(model, train_dataset, loss_func, optimizer)

Epoch [1/10], Training Loss: 7.9481
Epoch [2/10], Training Loss: 0.1413
Epoch [3/10], Training Loss: 0.1363
Epoch [4/10], Training Loss: 0.1315
Epoch [5/10], Training Loss: 0.1254
Epoch [6/10], Training Loss: 0.1171
Epoch [7/10], Training Loss: 0.1082
Epoch [8/10], Training Loss: 0.1048
Epoch [9/10], Training Loss: 0.1010
Epoch [10/10], Training Loss: 0.0966
Total training time: 135.11888003349304 seconds


In [ ]:
test(model, test_dataset, loss_func)

Validation Loss: 0.1828
Inference time: 1.2077627182006836 seconds


In [ ]:
gine_params = model_info['GCN']
model = GINE(**gine_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()

train(model, train_dataset, loss_func, optimizer)

Epoch [1/10], Training Loss: 120.5248
Epoch [2/10], Training Loss: 120.5248
Epoch [3/10], Training Loss: 120.5248
Epoch [4/10], Training Loss: 120.5248
Epoch [5/10], Training Loss: 120.5248


KeyboardInterrupt: 

In [ ]:
test(model, test_dataset, loss_func)

Validation Loss: 0.1827
Inference time: 0.1603529453277588 seconds


## Edge Prediction Models

In [14]:
dataset, info = EdgeRegressionDataset(node_features=config['node_features'],
                    edge_features=config['edge_features'],
                    **config['dataset_parameters']).load()

print(dataset[0])
print(type(dataset[0].x), dataset[0].x.shape)
print(type(dataset[0].edge_index), dataset[0].edge_index.shape)
print(type(dataset[0].edge_attr), dataset[0].edge_attr.shape)
print(type(dataset[0].y), dataset[0].y.shape)
print(info)

Data(x=[1268, 6], edge_index=[2, 5224], edge_attr=[5224, 8], y=[5224, 1], pos=[2, 1268])
<class 'torch.Tensor'> torch.Size([1268, 6])
<class 'torch.Tensor'> torch.Size([2, 5224])
<class 'torch.Tensor'> torch.Size([5224, 8])
<class 'torch.Tensor'> torch.Size([5224, 1])
{'num_static_node_features': 3, 'num_dynamic_node_features': 1, 'num_static_edge_features': 5, 'num_dynamic_edge_features': 1, 'previous_timesteps': 2}


In [15]:
num_train = int(len(dataset) * 0.8) # 80% train, 20% test

train_dataset = dataset[:num_train]
# train_loader = DataLoader(train_dataset) # batch_size=32, shuffle=True

test_dataset = dataset[num_train:]
# test_loader = DataLoader(test_dataset) # batch_size=32, shuffle=True

In [16]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
base_model_params = {
    'static_node_features': info['num_static_node_features'],
    'dynamic_node_features': info['num_dynamic_node_features'],
    'static_edge_features': info['num_static_edge_features'],
    'dynamic_edge_features': info['num_dynamic_edge_features'],
    'previous_timesteps': info['previous_timesteps'],
    'device': device,
}
lr_info = config['training_parameters']
model_info = config['model_parameters']

In [17]:
edge_gnn_params = model_info['EdgeGNN']
model = EdgeGNN(**edge_gnn_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()

train(model, train_dataset, loss_func, optimizer)

Epoch [1/10], Training Loss: 0.8729
Epoch [2/10], Training Loss: 0.1536
Epoch [3/10], Training Loss: 0.1536
Epoch [4/10], Training Loss: 0.1535
Epoch [5/10], Training Loss: 0.1535
Epoch [6/10], Training Loss: 0.1535
Epoch [7/10], Training Loss: 0.1535
Epoch [8/10], Training Loss: 0.1535
Epoch [9/10], Training Loss: 0.1535
Epoch [10/10], Training Loss: 0.1535
Total training time: 18.30726146697998 seconds


In [18]:
test(model, test_dataset, loss_func)

Validation Loss: 7.2946
Inference time: 0.1560046672821045 seconds


## Self-Supervised Learning Methods

In [ ]:
from tqdm import tqdm
from models.graph_mae2 import GraphMAE2
from utils.graph_mae2_utils import LinearRegression

In [ ]:
dataset, info = TemporalGraphDataset(node_features=config['node_features'],
                    edge_features=config['edge_features'],
                    **config['dataset_parameters']).load()

print(dataset[0])
print(type(dataset[0].x), dataset[0].x.shape)
print(type(dataset[0].edge_index), dataset[0].edge_index.shape)
print(type(dataset[0].edge_attr), dataset[0].edge_attr.shape)
print(type(dataset[0].y), dataset[0].y.shape)
print(info)

Data(x=[1268, 6], edge_index=[2, 5224], edge_attr=[5224, 8], y=[1268, 1], pos=[2, 1268])
<class 'torch.Tensor'> torch.Size([1268, 6])
<class 'torch.Tensor'> torch.Size([2, 5224])
<class 'torch.Tensor'> torch.Size([5224, 8])
<class 'torch.Tensor'> torch.Size([1268, 1])
{'num_static_node_features': 3, 'num_dynamic_node_features': 1, 'num_static_edge_features': 5, 'num_dynamic_edge_features': 1, 'previous_timesteps': 2}


In [ ]:
num_train = int(len(dataset) * 0.8) # 80% train, 20% test

train_dataset = dataset[:num_train]
# train_loader = DataLoader(train_dataset) # batch_size=32, shuffle=True

test_dataset = dataset[num_train:]
# test_loader = DataLoader(test_dataset) # batch_size=32, shuffle=True

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
base_model_params = {
    'static_node_features': info['num_static_node_features'],
    'dynamic_node_features': info['num_dynamic_node_features'],
    'static_edge_features': info['num_static_edge_features'],
    'dynamic_edge_features': info['num_dynamic_edge_features'],
    'previous_timesteps': info['previous_timesteps'],
    'device': device,
}
lr_info = config['training_parameters']
model_info = config['model_parameters']

In [ ]:
def pretrain(model, dataset, optimizer):
    start_time = time.time()

    max_epoch = 20
    epoch_iter = tqdm(range(max_epoch))
    model.to(device)
    for epoch in epoch_iter:
        model.train()
        running_loss = 0.0

        for graph in dataset:
            graph = graph.to(device)
            x = graph.x # Target
            target_nodes = torch.arange(x.shape[0], device=device, dtype=torch.long)

            optimizer.zero_grad()

            loss = model(graph, x, targets=target_nodes)
            running_loss += loss.item()

            loss.backward()
            optimizer.step()

        epoch_loss = running_loss / num_train
        epoch_iter.set_description(f"# Epoch {epoch}: train_loss: {epoch_loss:.4f}")

    end_time = time.time()
    print(f'Total pre-training time: {end_time - start_time} seconds')
    return model

def linear_probing(model, dataset, in_dim, out_dim, lr_f, weight_decay_f):
    start_time = time.time()

    # Should we freeze model parameters or fine-tune them?
    encoder = LinearRegression(in_dim, out_dim).to(device)

    num_finetune_params = [p.numel() for p in encoder.parameters() if  p.requires_grad]
    print(f"num parameters for finetuning: {sum(num_finetune_params)}")

    loss_f = torch.nn.L1Loss()
    optimizer_f = torch.optim.Adam(encoder.parameters(), lr=lr_f, weight_decay=weight_decay_f)

    best_model = None
    max_epoch_f = 50
    epoch_iter_f = tqdm(range(max_epoch_f))
    model.eval()
    encoder.train()
    for epoch in epoch_iter_f:
        running_loss = 0.0

        for graph in dataset:
            optimizer_f.zero_grad()

            graph = graph.to(device)
            with torch.no_grad():
                x = model.embed(graph)
                x = x.to(device)
            label = graph.y

            out = encoder(x)
            loss = loss_f(out, label)

            loss.backward()
            # torch.nn.utils.clip_grad_norm_(encoder.parameters(), max_norm=3)
            optimizer_f.step()
            running_loss += loss.item()

        epoch_loss = running_loss / num_train
        epoch_iter_f.set_description(f"# Epoch {epoch}: train_loss: {epoch_loss:.4f}")

    end_time = time.time()
    print('Final loss: ', epoch_loss)
    print(f'Total fine-tuning time: {end_time - start_time} seconds')

    return encoder

def test_ssl(model, ft_model, dataset, loss_func):
    start_time = time.time()
    model.eval()
    ft_model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for graph in dataset:
            graph = graph.to(device)
            x = model.embed(graph)
            labels = graph.y

            outputs = ft_model(x)

            loss = loss_func(outputs, labels)
            running_loss += loss.item()
    end_time = time.time()

    # Print validation statistics
    print(f'Validation Loss: {running_loss:.4f}')
    print(f'Inference time: {end_time - start_time} seconds')

In [ ]:
graphmae2_params = model_info['GRAPHMAE2']
in_dim = dataset[0].x.shape[1]
model = GraphMAE2(in_dim=in_dim, **graphmae2_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])

trained_model = pretrain(model, train_dataset, optimizer)

=== Use sce_loss and alpha_l=3 ===


# Epoch 19: train_loss: 0.0209: 100%|██████████| 20/20 [01:40<00:00,  5.02s/it]

Total pre-training time: 100.48602938652039 seconds


In [ ]:
out_dim = dataset[0].y.shape[1]
hidden_dim = graphmae2_params['num_hidden'] // graphmae2_params['nhead']
ft_model = linear_probing(trained_model, train_dataset, hidden_dim, out_dim, 0.0008, lr_info['weight_decay'])

num parameters for finetuning: 33


# Epoch 49: train_loss: 18.7243: 100%|██████████| 50/50 [01:09<00:00,  1.39s/it]

Final loss:  18.724270087569508
Total fine-tuning time: 69.55408716201782 seconds


In [ ]:
loss_func = torch.nn.L1Loss()
test_ssl(trained_model, ft_model, test_dataset, loss_func)

Validation Loss: 1028.6302
Inference time: 0.30884242057800293 seconds
